In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from scipy.stats import mode
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder

# 1. 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')

## 1. 데이터 전처리

In [ ]:
# Parch가 같은 사람들의 나이 평균으로 Age의 결측치 채우기
train['Age'] = train.groupby('Parch')['Age'].transform(lambda x: x.fillna(x.mean()))

# train에서 Age 결측치가 남아있다면 전체 평균으로 채우기
train['Age'].fillna(train['Age'].mean(), inplace=True)

# test에서도 동일한 처리
test['Age'] = test.groupby('Parch')['Age'].transform(lambda x: x.fillna(x.mean()))

# test에서 Age 결측치가 남아있다면 전체 평균으로 채우기
test['Age'].fillna(test['Age'].mean(), inplace=True)

# Fare 결측치를 Pclass가 같은 그룹의 평균으로 채우기
train['Fare'] = train.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.mean()))
test['Fare'] = test.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
# Cabin의 첫 글자 추출
train['Cabin_First_Letter'] = train['Cabin'].str[0]
test['Cabin_First_Letter'] = test['Cabin'].str[0]

# Cabin의 첫 글자를 Label Encoding으로 변환
# LabelEncoder를 사용하여 문자열을 숫자로 변환
encoder = LabelEncoder()
train['Cabin_First_Letter'] = encoder.fit_transform(train['Cabin_First_Letter'])
test['Cabin_First_Letter'] = encoder.transform(test['Cabin_First_Letter'])

# Cabin 컬럼 삭제
train.drop(columns=['Cabin'], inplace=True)
test.drop(columns=['Cabin'], inplace=True)

# 결측치를 Fare가 같은 그룹의 Cabin_First_Letter의 최빈값으로 대체하는 함수 정의
train['Cabin_First_Letter'] = train.groupby('Pclass')['Cabin_First_Letter'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 0))
test['Cabin_First_Letter'] = test.groupby('Pclass')['Cabin_First_Letter'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 0))

In [ ]:
# Embarked 열의 최빈값 계산
most_common_embarked_train = train['Embarked'].mode()[0]
most_common_embarked_test = test['Embarked'].mode()[0]

# Embarked 결측치를 최빈값으로 채우기
train['Embarked'].fillna(most_common_embarked_train, inplace=True)
test['Embarked'].fillna(most_common_embarked_test, inplace=True)

# 나이를 범주화하는 함수 정의
def categorize_age(age):
    if age <= 12:
        return 'Child'
    elif age <= 19:
        return 'Teenager'
    elif age <= 34:
        return 'Young Adult'
    elif age <= 49:
        return 'Middle Age'
    elif age <= 64:
        return 'Senior'
    else :
        return 'Elderly'

train['Age_Category'] = train['Age'].apply(categorize_age)
test['Age_Category'] = test['Age'].apply(categorize_age)

In [ ]:
# 성별 인코딩
train['Sex'] = train['Sex'].map({'male': 0, 'female': 1})
test['Sex'] = test['Sex'].map({'male': 0, 'female': 1})

# 나이 범주와 성별 결합
train['Age_Sex'] = train['Age_Category'] + '_' + train['Sex'].map({0: 'male', 1: 'female'})
test['Age_Sex'] = test['Age_Category'] + '_' + test['Sex'].map({0: 'male', 1: 'female'})

In [ ]:
# 나이를 범주화
train['Age_Category'] = train['Age'].apply(categorize_age)
test['Age_Category'] = test['Age'].apply(categorize_age)

# 불필요한 열 제거
train = train.drop(columns=['Name', 'Ticket', 'Age_Category'])
test = test.drop(columns=['Name', 'Ticket', 'Age_Category'])

In [ ]:
# 라벨 인코딩 함수
def label_encode_columns(df, columns):
    label_encoders = {}
    for col in columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
    return df, label_encoders

# 라벨 인코딩할 컬럼 지정
columns_to_encode = ['Embarked', 'Age_Sex']

# train 데이터와 test 데이터에 대해 라벨 인코딩 수행
train, train_label_encoders = label_encode_columns(train, columns_to_encode)
test, test_label_encoders = label_encode_columns(test, columns_to_encode)

# SibSp (형제/배우자 수)와 Parch (부모/자녀 수)를 합산한 후, 본인을 포함시켜 family_size 생성
train['family_size'] = train['SibSp'] + train['Parch'] + 1
test['family_size'] = test['SibSp'] + test['Parch'] + 1

In [ ]:
# 3. 학습/검증 데이터셋 분할
X = train.drop(columns=['Survived', 'PassengerId'])
y = train['Survived']

# 훈련/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define param_grid for hyperparameter tuning
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# 하이퍼파라미터 튜닝 설정 (GridSearchCV)
xgb_model = xgb.XGBClassifier(eval_metric="auc", use_label_encoder=False)

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='roc_auc',         # AUC 기준으로 최적 파라미터 탐색
    cv=5,                      # 5-폴드 교차 검증
    verbose=1,
    n_jobs=-1                  # 모든 CPU 코어 사용
)

# 4. 모델 튜닝 실행
grid_search.fit(X_train, y_train)

# 5. 최적 모델 학습 및 검증
best_model = grid_search.best_estimator_  # 최적 모델

# 최적 모델로 전체 훈련 데이터 학습
best_model.fit(X_train, y_train)

# 검증 데이터에 대한 예측
val_predictions = best_model.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, val_predictions)
print(f"Optimized AUC: {val_auc:.4f}")

# 6. 테스트 데이터에 대한 예측
X_test = test.drop(columns=['PassengerId'])  # PassengerId 제외
test_predictions = best_model.predict_proba(X_test)[:, 1]  # 예측 수행

# 결과를 데이터프레임으로 변환
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],  # 테스트 데이터의 PassengerId 가져오기
    'Survived': (test_predictions > 0.5).astype(int)  # 0.5 기준으로 생존 여부 예측
})

# 예측 결과 확인
print(submission.head())

# 7. 예측 결과를 CSV 파일로 저장 (캐글 제출용)
submission.to_csv('submission_basecode.csv', index=False)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Optimized AUC: 0.8940
   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         0


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:50:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:50:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
